In [12]:
import subprocess
import glob
import os
import numpy as np
import surface_tools as st

In [ ]:
#mapping volume blocks to cortical surface

In [19]:
outdir='../surface_data/area_labels'
voldir='../volume_data/area_blocks'
os.makedirs(outdir,exist_ok=True)
area_blocks=sorted(glob.glob('../volume_data/area_blocks/*.mnc'))
area_names=[]
for ab in area_blocks:
    ab=ab.replace('_block.mnc','')
    ab=ab.replace('../volume_data/area_blocks/','')
    area_names.append(ab)


area_names=sorted(area_names)
area_names.insert(0,'null')
np.savetxt(os.path.join(outdir,'area_labels.txt'),area_names,fmt='%s')

In [21]:
hemis=['right','left']
for k,area in enumerate(area_names[1:]):
    print(area)
    for hemi in hemis:
        mid_surface_name='../surface_data/obj_surfaces/mid_{}.obj'.format(hemi)

        subprocess.call('volume_object_evaluate -nearest_neighbour {} {} {}'.format(os.path.join(voldir,'{}_block.mnc'.format(area)),
                                                             mid_surface_name,
                                                             os.path.join(outdir,'{}_{}.txt'.format(area,hemi))),shell=True)
       

  
        
        

area6d_6d1
area6d_6d2
area6d_6d3
auditory_Te10
auditory_Te11
auditory_Te12
auditory_Te3
hippocampus_EC
ifs_ifj1
ifs_ifj2
ifs_ifs1
ifs_ifs2
ifs_ifs3
ifs_ifs4
pIPS_hIP4
pIPS_hIP5
pIPS_hIP6
pIPS_hIP7
pIPS_hIP8
pIPS_hOc6
pIPS_hPO1
sma_presma
sma_sma
sts_Te4
sts_Te5
v1
v2


In [ ]:
#tidy up individual areas
#TODO consider whether this is necessary to do, separate from the main tidying up.
mid_surface_name='../surface_data/obj_surfaces/mid_{}.obj'.format('left')
neighbours=st.get_neighbours(mid_surface_name)
hemis=['left','right']
for k,area in enumerate(area_names[1:]):
    print(area)
    
    for hemi in hemis:
       #fill holes
        area_txtfile=np.loadtxt(os.path.join(outdir,'{}_{}.txt'.format(area,hemi))).astype(bool)
        tidied=st.tidy_holes_binary(area_txtfile,neighbours, threshold_area=300,iterations=2)
        np.savetxt(os.path.join(outdir,'{}_{}_tidied.txt'.format(area,hemi)),tidied,fmt='%i')
      

area6d_6d1
area6d_6d2
area6d_6d3
auditory_Te10
auditory_Te11
auditory_Te12
auditory_Te3
hippocampus_EC


In [ ]:
#combine areas into single parcellation
reload(st)
padl_dir='/data1/bigbrain/PADL/data/raw'
hemis=['left','right']
for hemi in hemis:
    mid_surface_name='../surface_data/obj_surfaces/mid_{}.obj'.format(hemi)
    neighbours=st.get_neighbours(mid_surface_name)

    combined_areas = np.zeros(len(neighbours))
    overlaps = np.zeros(len(neighbours)).astype(bool)
    for k,area in enumerate(area_names[1:]):
        print(area)
        area_txtfile=np.loadtxt(os.path.join(outdir,'{}_{}.txt'.format(area,hemi)))
        overlaps+= np.logical_and(area_txtfile,combined_areas)
        combined_areas+= area_txtfile*area_names.index(area)
    tidied_combined=st.tidy_combined_atlas(combined_areas, overlaps, neighbours,threshold=300)
    
    np.savetxt(os.path.join(outdir,'combined_areas_{}_tidied.txt'.format(hemi)),tidied_combined, fmt='%i')
    np.savetxt(os.path.join(padl_dir,'combined_areas_{}_tidied.txt'.format(hemi)),tidied_combined, fmt='%i')
